In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from koopmaneigen.matrix_eigsolver import MatrixEigSolver

dictionary basis = $\{x, y , x^2, y^2, xy\}$

In [3]:
def normalize(v):
    return v/np.linalg.norm(v)

taking linear system $x_{n+1} = Ax_n$ where A is 
$\begin{bmatrix}
    0.9 & 0.3 \\
    0 & 0.6
\end{bmatrix}$

koopman eigenfunctions will be $\phi(x,y) = (x-y)$ with eigenvalue 0.9 and $\phi(x,y) = y$ with eigenvalue 0.6

### set up koopman matrix

In [4]:
lambda_1 = 0.9
lambda_2 = 0.6

In [5]:
### representing eigenfunctions in dictionary sopace

In [6]:
v_1 = [1,-1,0,0,0]; d_1 = lambda_1
v_2 = [0,0,1,1,-2]; d_2 = (lambda_1)**2
v_3 = [0,1,0,0,0]; d_3 = lambda_2
v_4 = [0,0,0,-1,1]; d_4 = lambda_1 * lambda_2
v_5 = [0,0,0,1,0]; d_5 = (lambda_2)**2

In [7]:
V = np.array([normalize(v) for v in [v_1,v_2, v_3, v_4, v_5]]).T
print(V)

[[ 0.70710678  0.          0.          0.          0.        ]
 [-0.70710678  0.          1.          0.          0.        ]
 [ 0.          0.40824829  0.          0.          0.        ]
 [ 0.          0.40824829  0.         -0.70710678  1.        ]
 [ 0.         -0.81649658  0.          0.70710678  0.        ]]


In [8]:
np.linalg.cond(V)

6.288429268634205

In [9]:
D = np.diag([d_1, d_2, d_3, d_4, d_5])
print(D)

[[0.9  0.   0.   0.   0.  ]
 [0.   0.81 0.   0.   0.  ]
 [0.   0.   0.6  0.   0.  ]
 [0.   0.   0.   0.54 0.  ]
 [0.   0.   0.   0.   0.36]]


In [10]:
np.linalg.inv(V)

array([[ 1.41421356,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.        , -0.        ,  2.44948974, -0.        , -0.        ],
       [ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [-0.        , -0.        ,  2.82842712, -0.        ,  1.41421356],
       [ 0.        ,  0.        ,  1.        ,  1.        ,  1.        ]])

In [11]:
# get transpose of koopman matrix
K_transpose = V @ D @ np.linalg.inv(V)
print(K_transpose)

[[ 0.9   0.    0.    0.    0.  ]
 [-0.3   0.6   0.    0.    0.  ]
 [ 0.    0.    0.81  0.    0.  ]
 [ 0.    0.    0.09  0.36 -0.18]
 [ 0.    0.   -0.54  0.    0.54]]


In [12]:
### looks like a tridiagonal matrix?

In [13]:
eigsolver = MatrixEigSolver(K_transpose)

### Power method on K_transpose

In [17]:
eigvec, eigvalue = eigsolver.power_iteration(K_transpose)
print(eigvec)
print(eigvalue)


[ 7.07106781e-01 -7.07106781e-01  8.25690676e-06  8.25690676e-06
 -1.65138135e-05]
0.8999999999631847


first eigenvector is calculated correctly by power iteration

### try assymetric power deflation algorithm

In [18]:
eigs = eigsolver.power_iteration_with_deflation_asymm(num_eigen=5, num_iterations = 100000, tolerance=1e-10) 
with np.printoptions(precision=4, suppress=True):
    for i, (eigvec,eigvalue) in enumerate(eigs):
        print(f"eigenvalue: {eigvalue: .4}")
        print(f"eigenvalue error: {eigvalue - D[i,i]:.4}")
        eigvec = eigvec.reshape(eigvec.shape[0])
        print(f"eigenvector norm error: {np.linalg.norm(eigvec - V[:,i]):.4}")
        print("--------")

eigenvalue:  0.9
eigenvalue error: -3.521e-10
eigenvector norm error: 6.254e-05
--------
eigenvalue:  0.81
eigenvalue error: -8.739e-11
eigenvector norm error: 4.247e-05
--------
eigenvalue:  0.6
eigenvalue error: -3.513e-10
eigenvector norm error: 7.563e-05
--------
eigenvalue:  0.54
eigenvalue error: 3.824e-10
eigenvector norm error: 4.052e-05
--------
eigenvalue:  0.36
eigenvalue error: 0.0
eigenvector norm error: 2.752e-09
--------
